this was pretty difficult for me. I have 4 separate loops:
<ol>
    <li> puts all rows from the original wikipedia table into a list.</li>
    <li>looks for "Not assigned" values and changes the values or removes rows accordingly.</li>
<li>looks for duplicate postal codes. It assigns each postal code as a key to a dict and assigns all neighborhoods within that postal code to the value as a set.</li>
    <li>goes back and lists all neighborhoods for each postal code.</li>
</ol>

In [30]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [105]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
        
row = 0
while row < len(res):
    if res[row][1] == "Not assigned":
        res.pop(row)
    elif res[row][2] == 'Not assigned':
        res[row][2] = res[row][1]
        row += 1
    else:
        row += 1
        
postal_code = {}
row = 0
while row < len(res):
    if res[row][0] in postal_code.keys():
        l = postal_code.get(res[row][0]);
        l.add(res[row][2])
        postal_code.update({res[row][0]:l})
        res.pop(row)
    else:
        l = {res[row][2]}
        postal_code.update({res[row][0]:l})
        row += 1
        
for row in range(len(res)):
    res[row][2] = postal_code.get(res[row][0])

pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])

,PostalCode,Borough,Neighborhood
0,M3A,North York,{Parkwoods}
1,M4A,North York,{Victoria Village}
2,M5A,Downtown Toronto,"{Regent Park, Harbourfront}"
3,M6A,North York,"{Lawrence Manor, Lawrence Heights}"
4,M7A,Queen's Park,{Queen's Park}
5,M9A,Etobicoke,{Islington Avenue}
6,M1B,Scarborough,"{Malvern, Rouge}"
7,M3B,North York,{Don Mills North}
8,M4B,East York,"{Woodbine Gardens, Parkview Hill}"
9,M5B,Downtown Toronto,"{Ryerson, Garden District}"


In [107]:
pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"]).shape

(103, 3)